## Cодержание:
* [First Bullet Header](#first-bullet)
* [Second Bullet Header](#second-bullet)

# Команда: Бета Банк


**Цель:** Создать CLTV модель, которая будет выдавать вероятности перехода в каждый из 17 продуктовых кластеров в течение 12 месяцев.

Альфа-Банком предоставлены следующие **данные**:

-   Тренировочный датасет `train_data.pqt` содержит данные о 200 000 клиентах банка и их целевых переменных за три последовательных месяца (month_1, month_2, month_3)
-   Тестовый датасет `test_data.pqt` записи о 100 000 клиентах за 3 последовательных месяца (month_4, month_5, month_6)
   
-   Продуктовый кластер, в котором клиент будет находится через год - `end_cluster`. Необходимо получить вероятности перехода клиента в продуктовые кластеры для последнего месяца (month_6).

  
-    Метрикой качества выступает **ROC-AUC**.

Данные о клиентах и масскированы.

## Подключение модулей

In [ ]:
!pip install catboost

In [ ]:
# работа с ОС
import os
import warnings
import time
# работа с данными
import json
import numpy as np
import pandas as pd
from typing import Optional, Dict, Tuple
# визуализация
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
from matplotlib.ticker import MaxNLocator
# работа с ML
from sklearn.utils import resample
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
# модели машинного обучения
# import optuna
from catboost import CatBoostClassifier, Pool
from lightgbm import LGBMClassifier
from sklearn.ensemble import StackingClassifier, RandomForestClassifier, ExtraTreesClassifier
# обработка ошибок
from pyarrow import ArrowInvalid
# настройка среды выполнения
warnings.filterwarnings("ignore")
pd.set_option('display.float_format', '{:.4f}'.format)
pd.set_option('display.max_rows', 93)

## Загрузка и изучение данных

In [ ]:
def read_df(filepath: str) -> Optional[pd.DataFrame]:
    """
    Функция для чтения датасета по указанному пути

    args:
    filepath (str) - путь к файлу .pqt / .parquet

    return:
    датасет в формате pd.DataFrame, если найден файл по указанному пути
    """
    if os.path.exists(filepath):
        try:
            result: pd.DataFrame = pd.read_parquet(filepath, engine='auto')
            print(f"Файл {filepath} успешно открыт")
            return result
        except ArrowInvalid:
            print("Неверный тип файла. Поддерживаемый - .pqt или .parquet")
            return
    print(f"Файл не найден по пути {filepath}")
    return

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# Путь до файла train_df
path_train_df = "/kaggle/input/alfa-hackaton/train_data.pqt"
# Путь до файла test_df
path_test_df = "/kaggle/input/alfa-hackaton/test_data.pqt"

# чтение файлов
train_df = read_df(path_train_df)
test_df = read_df(path_test_df)

# объединение файлов
df = pd.concat([train_df, test_df], ignore_index=True)

# меняем в test_df нумирацию месяца
df['date'] = df['date'].replace({'month_4': 'month_1', 'month_5': 'month_2', 'month_6': 'month_3'})


display(train_df.head(2))
display(test_df.head(2))

### Базовый анализ данных

In [ ]:
print(f"Количество записей в тренировочных данных: {len(train_df)}")
print(f"Количество записей в тестовых данных: {len(test_df)}")

В тестовой выборке у 30040 клиентов нет информации по 4 месяцу.

In [ ]:
df.dtypes.value_counts()

Признаки по большей части представляют собой числа с плавающей точкой, также присутствует 11 категориальных признаков, их необходимо закодировать.

In [ ]:
df[df.duplicated(subset=df.columns.difference(['id']), keep=False)].shape

Дубликаты записей есть. Удалние их в train не приводит к успеху.

In [ ]:
df['start_cluster'].value_counts()

Распределение основного признака (начального кластера, из которого будет осуществляться переход в конечный кластер, являющийся целевой переменной) сильно дизбалансное.

In [ ]:
df['end_cluster'].value_counts()

Распределение целевой переменной также сильно дизбалансное. В нашем решении, мы пробовали обучать на дизабалансом, баланс по мажорному классу, использовали SMOKE, баланс с указанием конкретного числа признаков, баланс к среднему числу признаков. Лучше всего работает на дизбалансном.

## Предобработка данных

### Восстановление численных признаков

In [ ]:
numeric_cols = df.drop(columns=['id']).select_dtypes(include=['number']).columns.tolist()
for col in numeric_cols:
    # получением минимального возможного значения признака
    mean_val = df[col].mean()
    # заполнение пропущенных значений минимальным значением признака
    df[col].fillna(mean_val, inplace=True)

Для начала были восстановленны пропущенные записи у всех численных признаков средними значениями признака. Пробовали также среднии с учетом категориального признака okved. Лучшее работает 1 способ.

In [ ]:
df['avg_a_oper_1m'] = df['sum_a_oper_1m'] / df['cnt_a_oper_1m']
df['avg_b_oper_1m'] = df['sum_b_oper_1m'] / df['cnt_b_oper_1m']
df['avg_c_oper_1m'] = df['sum_c_oper_1m'] / df['cnt_c_oper_1m']

df['avg_deb_d_oper_1m'] = df['sum_deb_d_oper_1m'] / df['cnt_deb_d_oper_1m']
df['avg_cred_d_oper_1m'] = df['sum_cred_d_oper_1m'] / df['cnt_cred_d_oper_1m']

df['avg_deb_e_oper_1m'] = df['sum_deb_e_oper_1m'] / df['cnt_deb_e_oper_1m']
df['avg_cred_e_oper_1m'] = df['sum_cred_e_oper_1m'] / df['cnt_cred_e_oper_1m']


df['avg_deb_f_oper_1m'] = df['sum_deb_f_oper_1m'] / df['cnt_deb_f_oper_1m']
df['avg_cred_f_oper_1m'] = df['sum_cred_f_oper_1m'] / df['cnt_cred_f_oper_1m']

df['avg_deb_g_oper_1m'] = df['sum_deb_g_oper_1m'] / df['cnt_deb_g_oper_1m']
df['avg_cred_g_oper_1m'] = df['sum_cred_g_oper_1m'] / df['cnt_cred_g_oper_1m']

df['avg_deb_h_oper_1m'] = df['sum_deb_h_oper_1m'] / df['cnt_deb_h_oper_1m']
df['avg_cred_h_oper_1m'] = df['sum_cred_h_oper_1m'] / df['cnt_cred_h_oper_1m']


df['avg_a_oper_3m'] = df['sum_a_oper_3m'] / df['cnt_a_oper_3m']
df['avg_b_oper_3m'] = df['sum_b_oper_3m'] / df['cnt_b_oper_3m']
df['avg_c_oper_3m'] = df['sum_c_oper_3m'] / df['cnt_c_oper_3m']

df['avg_deb_d_oper_3m'] = df['sum_deb_d_oper_3m'] / df['cnt_deb_d_oper_3m']
df['avg_cred_d_oper_3m'] = df['sum_cred_d_oper_3m'] / df['cnt_cred_d_oper_3m']

df['avg_deb_e_oper_3m'] = df['sum_deb_e_oper_3m'] / df['cnt_deb_e_oper_3m']
df['avg_cred_e_oper_3m'] = df['sum_cred_e_oper_3m'] / df['cnt_cred_e_oper_3m']

df['avg_deb_f_oper_3m'] = df['sum_deb_f_oper_3m'] / df['cnt_deb_f_oper_3m']
df['avg_cred_f_oper_3m'] = df['sum_cred_f_oper_3m'] / df['cnt_cred_f_oper_3m']

df['avg_deb_g_oper_3m'] = df['sum_deb_g_oper_3m'] / df['cnt_deb_g_oper_3m']
df['avg_cred_g_oper_3m'] = df['sum_cred_g_oper_3m'] / df['cnt_cred_g_oper_3m']

df['avg_deb_h_oper_3m'] = df['sum_deb_h_oper_3m'] / df['cnt_deb_h_oper_3m']
df['avg_cred_h_oper_3m'] = df['sum_cred_h_oper_3m'] / df['cnt_cred_h_oper_3m']

Введение дополнительного признака "среднего" по каждой операции каждого клиента за каждый месяц. Так как признак суммы имеет большое количество одного уникального значения. 

In [ ]:
def restore_cal(x: pd.Series) -> pd.Series:
    """
    Функция восстановления категориального признака последним значением для
    конкретного клиента

    args:
    x (pd.Series) - значения рассматриваемого признака в группе

    return:
    Возвращается группа с восстановленным пропущенным значением
    """
    # если есть пропущеные значения
    if x.isna().any() and not x.isna().all():
        # заполняем пропущенное значение последним в группе
        return x.fillna(x.dropna().iloc[-1])
    # если все пропуски, заполняем 'missing'
    elif x.isna().all():
        return x.fillna('missing')
    # если нет пропусков, возвращаем исходную группу
    return x

In [ ]:
cat_columns_to_restore = ['channel_code', 'city', 'city_type', 'ogrn_month', 'ogrn_year', 'okved', 'segment']
for column in cat_columns_to_restore:
    start_time = time.time()
    df[column] = df.groupby('id')[column].apply(restore_cal).reset_index()[column]
    end_time = time.time()
    print(f"Колонка - {column} - восстановлена за {end_time - start_time}")

У клиента категорильный признак такой же как и в других месяцах, кроме start_cluster и end_cluster.
Отсюда следует, что пропущенные категориальные признаки можно восстановить последним значением признака для клиента. Если вообще нету информации создаем класс - missing.

Шанс 3 одинаковых категориальных признаков.

channel_code - 99.98309597261547 %

city - 99.74894658491661 %

city_type - 99.315048321069 %

index_city_code - 99.59118119433495 %

ogrn_month - 99.9894837655631 %

ogrn_year - 99.98835702615914 %

okved - 99.61698352916413 %

segment - 99.80287835153736 %

start_cluster - 80.04299999999999 %

end_cluster - 88.4095 %




Еще из интересной статистики.

Шанс, что клиент остаётся в том же кластере с 2 по 3 месяц: 93.3%

In [ ]:
df

### Создание таблицы с 3 месяцами

In [ ]:
cat_cols = [
    "channel_code", "city", "city_type",
    "okved", "segment", "ogrn_month", "ogrn_year",
]
cat_cols_month_1 = [f'{col}_month_1' for col in cat_cols]
cat_cols_month_2 = [f'{col}_month_2' for col in cat_cols]
# сводная таблица по клиентам и месяцам
df = df.pivot_table(index='id', columns='date', aggfunc='first')
df.columns = [f'{col[0]}_{col[1]}' for col in df.columns]
df.reset_index(inplace=True)
df = df.drop(
    columns=['end_cluster_month_1', 'end_cluster_month_2'] + cat_cols_month_1 + cat_cols_month_2,
    axis=0)
categorical_columns = df.select_dtypes(include=['object']).columns

# заполняем пропущенные элементы новой категорией, так как у клиентов в тесте может не быть 4 месяца
df[categorical_columns] = df[categorical_columns].fillna("missing")

Самое главное проследить за изменениями признаков за эти 3 месяца. Создаем таблицу со всеми признаками за каждый месяц.

In [ ]:
df.head()

In [ ]:
# получение только численных признаков
numeric_cols = df.drop(columns=['id']).select_dtypes(include=['number']).columns.tolist()
for col in numeric_cols:
    # получением минимального возможного значения признака
    mean_val = df[col].mean()
    # заполнение пропущенных значений минимальным значением признака
    df[col].fillna(mean_val, inplace=True)

Заполняем опять средним. Появились nan так как у клиентов может не быть записи за 4 месяц.

### Восстановление start_claster с помощью Catboost 

Catboost лучше всего себя показал и быстрее, что не мало важно. Мы пробовали lightgbm, xgboost, нейронку pytorch, ансамбль из моделей ( Catboost, lightgbm, xgboost).

In [ ]:
# выделяем тренировочные данные (без пропущенных значений целевой переменной)
train_data = df[df['start_cluster_month_3'] != 'missing'].drop(
    ['id', 'end_cluster_month_3'], axis=1)
# выделяем тестовые данные (пропущенные значения целевой переменной)
predict_data = df[df['start_cluster_month_3'] == 'missing'].drop(
    ['id', 'end_cluster_month_3'], axis=1)

# получаем признаки для обучения и целевую переменную
X = train_data.drop('start_cluster_month_3', axis=1)
y = train_data['start_cluster_month_3']

# разбиваем на подвыборки обучения и тестирования 80/20
X_train, X_val, y_train, y_val = train_test_split(X, y,
                                                  test_size=0.2,
                                                  random_state=42)

#### Обучение CATBOOST

In [ ]:
# архитектура модели
catboost_model_start_cluster = CatBoostClassifier(
    iterations=1024,            # кол-во итераций обучения
    depth=6,                    # рекомендованная глубина модели
    learning_rate=0.075,        # скорость обучения
    random_seed=47,             # сид для воспроизводимости результата
    loss_function='MultiClass', # тип модели или функция ошибки
    task_type="GPU",            # обучение на видеокарте
    devices='0',
    early_stopping_rounds=20    # регуляризация ранней остановкой в случае
                                # отстутсвия изменения ф. ошибки 20 итераций
    )

In [ ]:
def train_catboost(model: CatBoostClassifier,
                   x_train: pd.DataFrame, y_train: pd.Series,
                   x_val: pd.DataFrame, y_val: pd.Series,
                   cat_names: pd.core.indexes.base.Index,
                   model_name: str,
                   verbose_step: int = 100) -> pd.DataFrame:
    """
    Функция обучения, тестирования и сохранения модели catboost

    args:
    model (CatBoostClassifier) - модель catboost
    x_train (pd.DataFrame) - датафрейм для обучения
    y_train (pd.Series) - целевая переменная в обучении
    x_val (pd.DataFrame) - датафрейм для тестирования точности модели
    y_train (pd.Series) - целевая переменная в тестировании
    cat_names (pd.core.indexes.base.Index) - список категориальных признаков
    из тренировочного и тестового датафреймов для автоматической предобработки
    самой моделью
    model_name (str) - имя модели для сохранения
    verbose_step (int) - шаг вывода статуса модели

    return:
    Возвращается важность признаков модели
    """
    model.fit(
        x_train, y_train,                   # обучающая выборка
        cat_features=np.array(cat_names),   # категориальные признаки
        eval_set=(x_val, y_val),            # тестовая выборка
        verbose=verbose_step                         # шаг вывода статуса модели
    )
    # сохранение модели
    model.save_model(f'{model_name}.json')
    # получение важности признаков
    feature_importance = model.get_feature_importance(prettified=True)
    return feature_importance

In [ ]:
# получаем список категориальных признаков
cat_names = X.select_dtypes(include=['object']).columns
# обучение модели с получением важности признаков
feature_importance = train_catboost(
    catboost_model_start_cluster, X_train, y_train, X_val, y_val, cat_names,
    'catboost_model_start_cluster')

In [ ]:
# предсказываем разные классы из тестовой подвыборки
y_pred = catboost_model_start_cluster.predict(X_val)
# статистика точности моделей по разным метрикам
print(classification_report(y_val, y_pred))

#### Восстанавление 6 месяца из тестовых данных

In [ ]:
X_predict = predict_data.drop('start_cluster_month_3', axis=1)
predicted_clusters = catboost_model_start_cluster.predict(X_predict)

In [ ]:
predicted_clusters_flat = np.ravel(predicted_clusters)
class_counts = pd.Series(predicted_clusters_flat).value_counts()
print(class_counts)

In [ ]:
predicted_index = 0

df_restore_start_cluster = df.copy()
for index, row in df_restore_start_cluster.iterrows():
    # Проверяем, содержится ли в столбце 'date' значение 'month6' и id >= 100000
    if row['id'] >= 200000:
        # Вставляем значение из серии в столбец 'start_cluster_month_3' текущей строки
        df_restore_start_cluster.at[index,
                                    'start_cluster_month_3'] = predicted_clusters[predicted_index][0]
        # Увеличиваем индекс текущей строки в серии
        predicted_index += 1

In [ ]:
matching_rows = df_restore_start_cluster[df_restore_start_cluster['id'] >= 200000].loc[(df_restore_start_cluster['start_cluster_month_1'] == df_restore_start_cluster['start_cluster_month_2']) & (
    df_restore_start_cluster['start_cluster_month_2'] == df_restore_start_cluster['start_cluster_month_3'])]
matching_rows

Также как и в train, у нас не у всех клиетов совпадают start_cluster'ы. 

## Обучение модели <a class="anchor" id="first-bullet"></a>

Также Catboost лучше всего себя показал и быстрее, что не мало важно. Мы пробовали lightgbm, xgboost, нейронку pytorch, ансамбль из моделей ( Catboost, lightgbm, xgboost).

### Подготовка данных

In [ ]:
# получаем обратно тренировочный и тестовый датасеты из общего
train_df = df_restore_start_cluster[df_restore_start_cluster['id']< 200000]
test_df = df_restore_start_cluster[df_restore_start_cluster['id'] >= 200000]
# получаем признаки для обучения и целевую переменную
X = train_df.drop(["id", "end_cluster_month_3"], axis=1)
y = train_df["end_cluster_month_3"]
# разбиваем на подвыборки обучения и тестирования 80/20
X_train, X_val, y_train, y_val = train_test_split(X, y,
                                                  test_size=0.2,
                                                  random_state=42)

In [ ]:
# архитектура модели
catboost_model_end_cluster = CatBoostClassifier(
    iterations=2025,            # кол-во итераций обучения
    depth=6,                    # рекомендованная глубина модели
    learning_rate=0.075,        # скорость обучения
    random_seed=47,             # сид для воспроизводимости результата
    loss_function='MultiClass', # тип модели или функция ошибки
    task_type="GPU",            # обучение на видеокарте
    devices='0',
    early_stopping_rounds=20    # регуляризация ранней остановкой в случае
                                # отстутсвия изменения функции ошибки 20
                                # итераций подряд
    )

In [ ]:
cat_names = X_train.select_dtypes(include=['object']).columns

feature_importance = train_catboost(
    catboost_model_end_cluster, X_train, y_train, X_val, y_val, cat_names,
    'catboost_model_end_cluster')

In [ ]:
feature_importance

In [ ]:
# предсказываем разные классы из тестовой подвыборки
y_pred = catboost_model_end_cluster.predict(X_val)
# статистика точности моделей по разным метрикам
print(classification_report(y_val, y_pred))

## Тестирование модели

In [ ]:
def weighted_roc_auc(y_true, y_pred, labels, weights_dict):
    unnorm_weights = np.array([weights_dict[label] for label in labels])
    weights = unnorm_weights / unnorm_weights.sum()
    classes_roc_auc = roc_auc_score(y_true, y_pred, labels=labels,
                                    multi_class="ovr", average=None)
    return sum(weights * classes_roc_auc)

In [ ]:
cluster_weights = pd.read_excel("/kaggle/input/alfa-hackaton/cluster_weights.xlsx").set_index("cluster")
weights_dict = cluster_weights["unnorm_weight"].to_dict()

In [ ]:
y_pred_proba = catboost_model_end_cluster.predict_proba(X_val)
weighted_roc_auc(y_val, y_pred_proba, catboost_model_end_cluster.classes_, weights_dict)

Прогноз на тестовой выборке

In [ ]:
sample_submission_df = pd.read_csv("/kaggle/input/alfa-hackaton/sample_submission.csv") # поменять на свой
last_m_test_df = test_df
last_m_test_df = last_m_test_df.drop(["id" , 'end_cluster_month_3'], axis=1)

pool2 = Pool(data=last_m_test_df, cat_features=np.array(cat_names))

test_pred_proba = catboost_model_end_cluster.predict_proba(pool2) # last_m_test_df
test_pred_proba_df = pd.DataFrame(test_pred_proba, columns=catboost_model_end_cluster.classes_)
sorted_classes = sorted(test_pred_proba_df.columns.to_list())
test_pred_proba_df = test_pred_proba_df[sorted_classes]

sample_submission_df[sorted_classes] = test_pred_proba_df
sample_submission_df.to_csv("final1.csv", index=False) # сохранение модели

In [ ]:
sample_submission_df

---
## Выводы

Мы решали **задачу прогнозирования временного ряда спроса товаров** собственного производства на 14 дней вперёд.

Заказчиком предоставлены исторические данные о **продажах за 1 год**, а также в закодированном виде товарная иерархия и информация о магазинах.  
Прогнозировалось **число проданных товаров в штуках  `pr_sales_in_units`** для каждого **SKU/товара** (2050 шт. в обучающей выборке) в каждом из **10 магазинов**.

Основные **закономерности**, выявленные в результате анализа:
- ***Годовой тренд***  - спад средних продаж в зимний сезон октябрь-март.
- ***Недельная сезонность*** - пик продаж в субботу, спад в понедельник.
- В течение года несколько высоких ***пиков спроса, в основном в районе праздников***. Самые резкие подъёмы продаж в период Нового года и Пасхи. Подъем продаж начинается за несколько дней до.
- 40,6% записей относятся к продажам по промоакциям. Возможны одновременные продажи товара в одном магазине по промо и без.
- В данных представлены продукты с ***неполными временными рядами***: продавались только в дни около Пасхи, начали продаваться полгода назад.
- Во всех магазинах разный ассортимент товаров даже при условии одинаковых характеристик торговой точки.
- Все мета-признаки как характеристики магазинов и товаров показали влияние на средний спрос

На основе имеющихся данных **сгенерированы новые признаки:**  
- Календарные: день недели, число месяца, номер недели, флаг выходного дня (взят из доп. таблицы)
- Лаговые признаки 1-30 дней
- Скользящее среднее за 7 и 14 предыдущих дней
- Кластеризация по характеристикам магазинов и товаров
    
Чтобы временные ряды каждой комбинации Магазин-Товар были полными создан новый датасет, в который добавлены отсутствующие даты с нулевыми продажами.

 Обучение, валидация и выбор лучшего набора гиперпараметров проводится на **кросс-валидации Walk Forward**: подбор гиперпараметров на фолде проводится на valid-выборке, оценка лучшей модели на фолде на test-выборке.   
В итоге выбрана одна модель среди лучших на каждом фолде.

 Предсказание спроса обученной моделью делается последовательно на каждый следующий день с промежуточным перерасчётом лаговых признаков (учитывается предсказанное значение спроса в предыдущий день).

Метрика качества  **ROC-AUC**.  

Лучший результат по качеству и скорости показала модель градиентного бустинга **Catboost**.  <br>
Полученный результат: ROC-AUC = **0.903898**, на всей выборке. <br>
Полученный результат: ROC-AUC = **0.896941145**, на паблик выборке (1 место).



